In [1]:
! pip install tdqm


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [28]:
! pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 10.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 11.7 MB/s eta 0:00:00m eta 0:00:010:01:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
import scipy
from tqdm import tqdm
from xgboost import XGBClassifier

In [1]:
df_posts = pd.read_pickle(r"posts_charlie_hebdo.pkl")
df_replies= pd.read_pickle(r"replies_charlie_hebdo.pkl")

NameError: name 'pd' is not defined

In [4]:
# Define features and target
X_text = df_posts['text']
X_numeric = df_posts[['followers', 'favorite_count', 'retweet_count', 'verified', 'replies', 'first_time_diff']]
y = df_posts['rumour']

In [5]:
# Split the data into training and testing sets
X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(
    X_text, X_numeric, y, test_size=0.2, random_state=42,stratify=y
)

In [6]:
# Vectorize text data
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

In [7]:
X_train_tfidf

<1601x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 21339 stored elements in Compressed Sparse Row format>

In [8]:


# Initialize the Robust Scaler
scaler = RobustScaler()

X_train_numeric_scaled  = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

In [9]:
X_train_combined = scipy.sparse.hstack([X_train_tfidf, X_train_numeric_scaled])
X_test_combined = scipy.sparse.hstack([X_test_tfidf, X_test_numeric_scaled])

In [10]:
X_train_combined

<1601x1006 sparse matrix of type '<class 'numpy.float64'>'
	with 29946 stored elements in COOrdinate format>

In [11]:
# Initialize and train SVM classifier
svm_classifier = SVC()
svm_classifier.fit(X_train_combined, y_train)
svm_pred = svm_classifier.predict(X_test_combined)
svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred, average='macro')
svm_recall = recall_score(y_test, svm_pred, average='macro')

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:

# Initialize and train Random Forest classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_combined, y_train)
rf_pred = rf_classifier.predict(X_test_combined)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred, average='macro')
rf_recall = recall_score(y_test, rf_pred, average='macro')

In [13]:
# Initialize and train Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(X_train_combined, y_train)
gb_pred = gb_classifier.predict(X_test_combined)
gb_accuracy = accuracy_score(y_test, gb_pred)
gb_precision = precision_score(y_test, gb_pred, average='macro')
gb_recall = recall_score(y_test, gb_pred, average='macro')

In [14]:
print("SVM Accuracy:", svm_accuracy)
print("SVM Precision:", svm_precision)
print("SVM Recall:", svm_recall)
print("RF Accuracy:", rf_accuracy)
print("RF Precision:", rf_precision)
print("RF Recall:", rf_recall)
print("GB Accuracy:", gb_accuracy)
print("GB Precision:", gb_precision)
print("GB Recall:", gb_recall)

SVM Accuracy: 0.7755610972568578
SVM Precision: 0.3877805486284289
SVM Recall: 0.5
RF Accuracy: 0.8778054862842892
RF Precision: 0.8352773769440436
RF Recall: 0.7988388710253662
GB Accuracy: 0.8603491271820449
GB Precision: 0.8079959514170041
GB Recall: 0.7717934976777421


In [15]:
# Define the model
gbc = GradientBoostingClassifier()

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'min_samples_split': [2, 5, 10]
}

# Setup the grid search
grid_search = GridSearchCV(estimator=gbc, param_grid=param_grid, cv=5, scoring='f1', verbose=2, n_jobs=-1)

# Perform the grid search
grid_search.fit(X_train_combined, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train the best model
best_gbc = grid_search.best_estimator_
best_gbc.fit(X_train_combined, y_train)

# Predict and evaluate
y_pred = best_gbc.predict(X_test_combined)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] END learning_rate=0.01, max_depth=3, min_samples_split=2, n_estimators=50, subsample=0.7; total time=   0.5s
[CV] END learning_rate=0.01, max_depth=3, min_samples_split=2, n_estimators=50, subsample=0.7; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=3, min_samples_split=2, n_estimators=50, subsample=0.8; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=3, min_samples_split=2, n_estimators=50, subsample=0.8; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=3, min_samples_split=2, n_estimators=50, subsample=0.8; total time=   0.6s
[CV] END learning_rate=0.01, max_depth=3, min_samples_split=2, n_estimators=50, subsample=0.9; total time=   0.5s
[CV] END learning_rate=0.01, max_depth=3, min_samples_split=2, n_estimators=50, subsample=0.9; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=3, min_samples_split=2, n_estimators=100, subsample=0.7; total time=   0.7s
[CV] END learning_rate=0

In [16]:
best_params

{'learning_rate': 0.1,
 'max_depth': 7,
 'min_samples_split': 2,
 'n_estimators': 100,
 'subsample': 0.7}

In [25]:
# Initialize and train Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier(**best_params)
gb_classifier.fit(X_train_combined, y_train)
gb_pred = gb_classifier.predict(X_test_combined)
gb_accuracy = accuracy_score(y_test, gb_pred)
gb_precision = precision_score(y_test, gb_pred, average='macro')
gb_recall = recall_score(y_test, gb_pred, average='macro')

In [26]:
print("GB Accuracy:", gb_accuracy)
print("GB Precision:", gb_precision)
print("GB Recall:", gb_recall)

GB Accuracy: 0.8802992518703242
GB Precision: 0.8293820795817601
GB Recall: 0.8241336191496963


In [33]:

xgb = XGBClassifier(eval_metric='logloss')

# Train the best model
xgb.fit(X_train_combined, y_train)

# Predict and evaluate
xgb_pred = xgb.predict(X_test_combined)
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred, average='macro')
xgb_recall = recall_score(y_test, xgb_pred, average='macro')

In [35]:
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)

XGB Accuracy: 0.8653366583541147
XGB Precision: 0.8111667558681857
XGB Recall: 0.790800285816363


In [ ]:
# Define the model
xgb = XGBClassifier(eval_metric='logloss')

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9, 1.0]
    #'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    #'min_child_weight': [1, 3, 5]
}

# Create a StratifiedKFold object with tqdm progress bar
cv = StratifiedKFold(n_splits=5)
cv_splits = list(cv.split(X_train_combined, y_train))
tqdm_cv_splits = tqdm(cv_splits, total=len(cv_splits), desc="Grid Search Progress")

# Setup the grid search with the tqdm progress bar
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=tqdm_cv_splits, scoring='f1', verbose=2, n_jobs=-1)

# Perform the grid search
grid_search.fit(X_train_combined, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train the best model
best_xgb = grid_search.best_estimator_
best_xgb.fit(X_train_combined, y_train)

# Predict and evaluate
y_pred = best_xgb.predict(X_test_combined)
print(classification_report(y_test, y_pred, zero_division=1))

Grid Search Progress: 100%|█████████████████████████████| 5/5 [00:00<00:00, 2305.07it/s]

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.7; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.7; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=

In [ ]:
best_params